In [ ]:
dataset=spark.read.csv("C:/Users/MVJ/Desktop/SEYNSE/Exam. codes/solar - pipelines/sonarall-data.csv", header=True)

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler

In [1]:
stages = [] # stages in our Pipeline

In [2]:
features= ['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22',
 'V23','V24','V25','V26','V27','V28','V29','V30','V31','V32','V33','V34','V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41',
 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58',
 'V59', 'V60']

In [3]:
data=dataset.select(dataset.V1.cast('float'),dataset.V2.cast('float'),dataset.V3.cast('float'),dataset.V4.cast('float'),
dataset.V5.cast('float'),dataset.V6.cast('float'),dataset.V7.cast('float'),dataset.V8.cast('float'),
dataset.V9.cast('float'),dataset.V10.cast('float'),dataset.V11.cast('float'),dataset.V12.cast('float'),
dataset.V13.cast('float'),dataset.V14.cast('float'),dataset.V15.cast('float'),dataset.V16.cast('float'),
dataset.V17.cast('float'),dataset.V18.cast('float'),dataset.V19.cast('float'),dataset.V20.cast('float'),
dataset.V21.cast('float'),dataset.V22.cast('float'),dataset.V23.cast('float'),dataset.V24.cast('float'),
dataset.V25.cast('float'),dataset.V26.cast('float'),dataset.V27.cast('float'),dataset.V28.cast('float'),
dataset.V29.cast('float'),dataset.V30.cast('float'),dataset.V31.cast('float'),dataset.V32.cast('float'),
dataset.V33.cast('float'),dataset.V34.cast('float'),dataset.V35.cast('float'),dataset.V36.cast('float'),
dataset.V37.cast('float'),dataset.V38.cast('float'),dataset.V39.cast('float'),dataset.V40.cast('float'),
dataset.V41.cast('float'),dataset.V42.cast('float'),dataset.V43.cast('float'),dataset.V44.cast('float'),
dataset.V45.cast('float'),dataset.V46.cast('float'),dataset.V47.cast('float'),dataset.V48.cast('float'),
dataset.V49.cast('float'),dataset.V50.cast('float'),dataset.V51.cast('float'),dataset.V52.cast('float'),
dataset.V53.cast('float'),dataset.V54.cast('float'),dataset.V55.cast('float'),dataset.V56.cast('float'),
dataset.V57.cast('float'),dataset.V58.cast('float'),dataset.V59.cast('float'),dataset.V60.cast('float'),
dataset.Class.cast('String'))

NameError: name 'dataset' is not defined

In [6]:
label_stringIdx = StringIndexer(inputCol="Class", outputCol="label")
stages += [label_stringIdx]

assembler = VectorAssembler(inputCols=features, outputCol="features")
stages += [assembler]

In [7]:
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(data)
data2 = pipelineModel.transform(data)

In [8]:
selectedcols = ["label", "features"]
dataset2 = data2.select(selectedcols)
display(dataset2)

DataFrame[label: double, features: vector]

In [9]:
## dividing dataset into training and test

(trainingData, testData) = dataset2.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

145
63


In [41]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)


In [11]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

In [12]:
predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [13]:
selected = predictions.select("label", "prediction", "probability")
display(selected)

DataFrame[label: double, prediction: double, probability: vector]

In [14]:
## AUC calculations
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator_forAUC = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_forAUC.evaluate(predictions)

0.8438775510204082

In [25]:
###Accuracy calculations
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_forAccuracy= MulticlassClassificationEvaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol('prediction')
evaluator_forAccuracy.getMetricName()
accuracy=evaluator_forAccuracy.evaluate(predictions)


In [37]:
tn=predictions.where((predictions.label == 0) & (predictions.prediction == 0)).count()
tp=predictions.where((predictions.label == 1) & (predictions.prediction == 1)).count()
fp=predictions.where((predictions.label == 0) & (predictions.prediction == 1)).count()
fn=predictions.where((predictions.label == 1) & (predictions.prediction == 0)).count()

In [123]:
print("Accuracy: "+str(accuracy))
print ("True Positives: "+str(tp))
print ("True Negatives: "+str(tn))
print ("False Positives: "+str(fp))
print ("False Negatives: "+str(fn))
print ("Total: "+str(predictions.count()))
precisionn=(tp/(tp+fp))
recalll=(tp/(tp+fn))
acc=((tp+tn)/(tp+tn+fp+fn))
print ("Precision: "+str(precisionn))
print ("Recall: "+str(recalll))
print ("Accuracy2: "+str(acc))


Accuracy: 0.7777777777777778
True Positives: 19
True Negatives: 30
False Positives: 5
False Negatives: 9
Total: 63
Precision: 0.7916666666666666
Recall: 0.6785714285714286
Accuracy2: 0.7777777777777778


## Using GridSearch

In [40]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw p

In [43]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])  
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])# check elastic-net regularized generalized linear models
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

cv=CrossValidator(estimator=lr,estimatorParamMaps=paramGrid,evaluator=evaluator_forAccuracy,numFolds=3)
cvModel=cv.fit(trainingData)

In [125]:
predictionsGrid=cvModel.transform(testData)
evaluator_forAccuracy.evaluate(predictionsGrid)
print('Model Intercept: ', cvModel.bestModel.explainParam)


Model Intercept:  <bound method Params.explainParam of LogisticRegression_4375906eb5ca7b8014b5>


In [117]:
c=predictions.select("label", "prediction", "probability")
display(c)

DataFrame[label: double, prediction: double, probability: vector]

In [119]:
c.show()

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  0.0|       0.0|[0.79251455926873...|
|  0.0|       0.0|[0.88260916778977...|
|  0.0|       0.0|[0.99941025693265...|
|  0.0|       0.0|[0.99965125729542...|
|  0.0|       0.0|[0.99519082669886...|
|  0.0|       0.0|[0.99651076556990...|
|  0.0|       0.0|[0.78160932250987...|
|  0.0|       0.0|[0.69623554562432...|
|  0.0|       0.0|[0.84998646337528...|
|  0.0|       1.0|[0.49439364674981...|
|  0.0|       0.0|[0.70554107154329...|
|  0.0|       1.0|[0.04285436175769...|
|  0.0|       0.0|[0.66320103269790...|
|  0.0|       0.0|[0.52603745910243...|
|  0.0|       0.0|[0.91257633775016...|
|  0.0|       0.0|[0.73220159643735...|
|  0.0|       0.0|[0.99355091149573...|
|  0.0|       0.0|[0.80909402144310...|
|  0.0|       0.0|[0.93159655086247...|
|  0.0|       0.0|[0.99907299350773...|
+-----+----------+--------------------+
only showing top 20 rows



In [128]:
tnGrid=predictionsGrid.where((predictionsGrid.label == 0) & (predictionsGrid.prediction == 0)).count()
tpGrid=predictionsGrid.where((predictionsGrid.label == 1) & (predictionsGrid.prediction == 1)).count()
fpGrid=predictionsGrid.where((predictionsGrid.label == 0) & (predictionsGrid.prediction == 1)).count()
fnGrid=predictionsGrid.where((predictionsGrid.label == 1) & (predictionsGrid.prediction == 0)).count()

In [129]:
print ("True Positives: "+str(tpGrid))
print ("True Negatives: "+str(tnGrid))
print ("False Positives: "+str(fpGrid))
print ("False Negatives: "+str(fnGrid))
print ("Total: "+str(predictionsGrid.count()))
precisionn=(tpGrid/(tpGrid+fpGrid))
recalll=(tpGrid/(tpGrid+fnGrid))
acc=((tpGrid+tnGrid)/(tpGrid+tnGrid+fpGrid+fnGrid))
print ("Precision: "+str(precisionn))
print ("Recall: "+str(recalll))
print ("Accuracy2: "+str(acc))

True Positives: 19
True Negatives: 30
False Positives: 5
False Negatives: 9
Total: 63
Precision: 0.7916666666666666
Recall: 0.6785714285714286
Accuracy2: 0.7777777777777778
